### **Notebook 4: Delta Live Tables for Incremental Updates**

#### Steps:
1. Use Bronze and Silver Delta tables as input for DLT.
2. Update the Silver and Gold layers incrementally.

#### Code:

In [ ]:
import dlt
from pyspark.sql.functions import col, sum

# Bronze Table
@dlt.table
def bronze_retail_data():
    return spark.read.format("delta").load("/mnt/s3data/bronze/retail_data")

# Silver Table
@dlt.table
def silver_retail_data():
    return dlt.read("bronze_retail_data").filter(col("transaction_amount").isNotNull())

# Gold Table
@dlt.table
def gold_retail_data():
    existing_gold = spark.read.format("delta").load("/mnt/s3data/gold/retail_data")
    new_aggregates = (dlt.read("silver_retail_data")
                      .groupBy("region", "year")
                      .agg(sum("transaction_amount").alias("total_sales")))
    return existing_gold.union(new_aggregates)

---